In [122]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf ##tensorflow1을 기반으로 작성하여 나중에 2로 구현해보기.
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [45]:
def min_max_scaling(x): #0~1 값으로 정규화.
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7) #1e-7은 0으로 나누는 오류 예방.

def reverse_scaling(org_x, x): #정규화 된 값을 되돌림.
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

def remove_comma(x):
    return x.replace(',','')

In [116]:
#############하이퍼 파라미터
##5일별로 batch설정하여 추가 1일 데이터 예측.
seq_len = 5 #n_step
sequence_length = seq_len + 1

n_input = 2

In [95]:
####################READ STOCK DATA#########################
stock_data = pd.read_csv('005930.KS.csv')
stock = stock_data.drop([stock_data.columns[1],stock_data.columns[2],stock_data.columns[3],stock_data.columns[5],stock_data.columns[6]],axis=1)
stock = stock.dropna(axis=0) ##결측치가 있는 행 제거.(2019.11.14, 19.12.31이 비어 임의로 채움.)
print(stock, stock.shape)

###################READ EXCHANGE RATE DATA######################
exchange_data = pd.read_csv('exchange_rate.csv')
exchange_data = exchange_data.dropna(axis=0) #결측치가 있는 행 제거.(주말,공휴일)
exchange_data['Won/Dollar'] = exchange_data['Won/Dollar'].apply(remove_comma) #문자열의 ,를 삭제. apply()는 함수를 로우 또는 칼럼에 적용시켜줌. 
exchange_data = exchange_data.astype({'Won/Dollar':np.float}) #문자열인 환율 데이터를 부동소수점으로 변환
print(exchange_data)
print(exchange_data['Won/Dollar'].dtypes)

           Date  Close
0    2019-03-04  44850
1    2019-03-05  44250
2    2019-03-06  44000
3    2019-03-07  44450
4    2019-03-08  43800
..          ...    ...
244  2020-02-25  57900
245  2020-02-26  56500
246  2020-02-27  55900
247  2020-02-28  54200
248  2020-03-02  55000

[249 rows x 2 columns] (249, 2)
           Date  Won/Dollar
0    2019-03-04      1119.9
1    2019-03-05      1125.0
2    2019-03-06      1125.8
3    2019-03-07      1127.8
4    2019-03-08      1128.3
..          ...         ...
358  2020-02-25      1217.7
359  2020-02-26      1212.7
360  2020-02-27      1216.8
361  2020-02-28      1215.9
364  2020-03-02      1215.3

[249 rows x 2 columns]
float64


In [102]:

stock_close = stock['Close'].values.reshape(len(stock_close),1) #종가 값만을 저장.
#stock_close = stock.values[:,-1]
result_stock = []
for index in range(len(stock_close) - sequence_length):
    result_stock.append(stock_close[index:index+sequence_length])
#5+1 개씩 배열로 저장. 즉, result는 2차원배열.
print(stock_close,stock_close.shape)

exchange = exchange_data['Won/Dollar'].values.reshape(len(exchange),1) # 환율 값 저장.
result_exchange = []
for index in range(len(exchange) - sequence_length):
    result_exchange.append(exchange[index:index + sequence_length])

print(exchange)

[[44850]
 [44250]
 [44000]
 [44450]
 [43800]
 [43650]
 [44650]
 [43850]
 [43850]
 [44200]
 [43700]
 [43900]
 [44050]
 [45850]
 [46550]
 [45500]
 [45250]
 [45350]
 [44850]
 [44650]
 [45050]
 [45750]
 [46600]
 [46950]
 [46850]
 [46650]
 [46650]
 [46700]
 [46250]
 [46850]
 [47050]
 [47250]
 [47050]
 [45600]
 [45300]
 [45350]
 [45200]
 [44750]
 [44650]
 [44850]
 [46150]
 [45850]
 [45900]
 [45300]
 [44850]
 [44250]
 [42450]
 [42900]
 [42650]
 [42650]
 [42550]
 [41550]
 [41200]
 [42000]
 [43150]
 [43500]
 [43850]
 [42700]
 [42650]
 [42550]
 [41800]
 [42550]
 [42500]
 [43800]
 [43450]
 [43900]
 [44200]
 [44800]
 [44850]
 [44600]
 [43750]
 [44000]
 [43900]
 [44350]
 [45350]
 [45500]
 [45700]
 [45500]
 [45600]
 [45700]
 [46500]
 [47000]
 [46600]
 [46250]
 [45400]
 [46000]
 [45650]
 [44400]
 [45100]
 [45550]
 [46200]
 [46300]
 [46450]
 [46850]
 [46050]
 [46100]
 [46800]
 [47200]
 [47300]
 [46400]
 [47200]
 [47150]
 [46100]
 [46550]
 [45350]
 [45200]
 [44950]
 [43950]
 [43500]
 [43200]
 [42650]
 

In [103]:
#Normalize Data
norm_stock = min_max_scaling(stock_close)
print(norm_stock,shape(norm_stock))

norm_exchange = min_max_scaling(exchange)
print(norm_exchange,norm_exchange.shape)

[[0.17216981]
 [0.14386792]
 [0.13207547]
 [0.15330189]
 [0.12264151]
 [0.11556604]
 [0.16273585]
 [0.125     ]
 [0.125     ]
 [0.14150943]
 [0.11792453]
 [0.12735849]
 [0.13443396]
 [0.21933962]
 [0.25235849]
 [0.20283019]
 [0.19103774]
 [0.19575472]
 [0.17216981]
 [0.16273585]
 [0.18160377]
 [0.21462264]
 [0.25471698]
 [0.27122642]
 [0.26650943]
 [0.25707547]
 [0.25707547]
 [0.25943396]
 [0.23820755]
 [0.26650943]
 [0.2759434 ]
 [0.28537736]
 [0.2759434 ]
 [0.20754717]
 [0.19339623]
 [0.19575472]
 [0.18867925]
 [0.16745283]
 [0.16273585]
 [0.17216981]
 [0.23349057]
 [0.21933962]
 [0.22169811]
 [0.19339623]
 [0.17216981]
 [0.14386792]
 [0.05896226]
 [0.08018868]
 [0.06839623]
 [0.06839623]
 [0.06367925]
 [0.01650943]
 [0.        ]
 [0.03773585]
 [0.09198113]
 [0.10849057]
 [0.125     ]
 [0.07075472]
 [0.06839623]
 [0.06367925]
 [0.02830189]
 [0.06367925]
 [0.06132075]
 [0.12264151]
 [0.10613208]
 [0.12735849]
 [0.14150943]
 [0.16981132]
 [0.17216981]
 [0.16037736]
 [0.12028302]
 [0.13

In [110]:
#label 데이터와 x데이터 만들기.
#####의문 : 환율과 주가를 세로로 합쳐 하나의 입력으로 하는게 좋나, 두 입력으로 하는게 좋은지. => 전자!!!
x = np.concatenate((norm_stock, norm_exchange), axis=1) #axis=1, 세로로 합친다.
print("x.shape:", x.shape)
print("x[0]:", x[0])

y = x[:,0] #label은 종가로 한다.
print("y[0],y[-1],len(y):",y[0],y[-1],len(y)) #첫날과 끝날의 종가.

dataX = [] #입력으로 사용될 sequence data.
dataY = [] #출력으로 사용.
for i in range(0,len(y) - sequence_length):
    _x = x[i:i + seq_len]
    _y = y[i + seq_len] #seq_len 기반으로 다음 날 나타날 종가.
    if i is 0:
        print(_x ,"->", _y) #첫번째만 출력.
    dataX.append(_x) #주가데이터리스트 생성
    dataY.append(_y) #종가리스트 생성
    
#split train and test data.
#전체 70%를 학습용으로 사용.
train_size = int(len(dataY) * 0.7)
#나머지 30%를 테스트용으로 사용.
test_size = len(dataY) - train_size

#데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

#데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX1[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])



x.shape: (249, 2)
x[0]: [0.17216981 0.        ]
y[0],y[-1],len(y): 0.17216981131994258 0.6509433962233446 249
[[0.17216981 0.        ]
 [0.14386792 0.05151515]
 [0.13207547 0.05959596]
 [0.15330189 0.07979798]
 [0.12264151 0.08484848]] -> 0.11556603773530393


In [125]:
#텐서플로우 플레이스홀더 생성
#입력 X, 출력 Y를 생성.
X = tf.placeholder(tf.float32,[None, seq_len, n_input]) ## each is [batch_sizeX, TimesX, input vector size]
Y = tf.placeholder(tf.float32, [None, 1]) #n_class: 1 , 결과는 종가.
print("X:", X)
print("Y:", Y)

#검증용 측정지표를 산출하기 위한 targets, predictions를 생성.
targets = tf.placeholder(tf.float32,[None,1])
print("targets:", targets)

predictions = tf.placeholder(tf.float32, [None,1])
print("predictions:", predictions)

X: Tensor("Placeholder_3:0", shape=(?, 5, 2), dtype=float32)
Y: Tensor("Placeholder_4:0", shape=(?, 1), dtype=float32)
targets: Tensor("Placeholder_5:0", shape=(?, 1), dtype=float32)
predictions: Tensor("Placeholder_6:0", shape=(?, 1), dtype=float32)


In [ ]:
#모델(LSTM 네트워크) 생성
def lstm_cell(): #atom에서 수업시간에 한 lstm과 어떻게 다른지 구분.
    

In [ ]:
#두 가지 입력으로 lstm 가능?